# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
print("\t Current Working Directory: ", os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

print("Total # of *.csv files found: ", len(file_path_list))

	 Current Working Directory:  /home/workspace
Total # of *.csv files found:  30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line)

print("Total # of rows: ", len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

Total # of rows:  8056


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling in Apache Cassandra

## The **event_datafile_new.csv** contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the **event_datafile_new.csv** after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [10]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Established !!")
except Exception as e:
    print(f"Connection Failed !! Error : {e}")

Connection Established !!


#### Create Keyspace

In [12]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS song_plays
                       WITH replication = {'class': 'SimpleStrategy',
                                           'replication_factor' : 1};""")
except Exception as e:
    print(f"Error creating keyspace: {e}")

#### Set Keyspace

In [13]:
session.set_keyspace('song_plays')

## Query 1:
#### Query Requirement: 
Give the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4 

#### Query Requirement analysis: 
Since the query requires filtering on based `sessionId` and `itemInSession`, primary key must have these columns. 
- PARTITION KEY - `sessionId` 
- CLUSTURING COLUMNS - `itemInSession` 
- PRIMARY KEY - `(sessionId, itemInSession)` 

#### SELECT statement: 
```SELECT artist, song, length
FROM session_song_details
WHERE sessionId = 338
  AND itemInSession = 4;```

In [16]:
session_song_create_query = """CREATE TABLE IF NOT EXISTS session_song_details (
                                   sessionId INT,
                                   itemInSession INT,
                                   artist TEXT,
                                   song TEXT,
                                   length FLOAT,
                                   PRIMARY KEY (sessionId, itemInSession));
                                """
try:
    session.execute(session_song_create_query)
    print("Table Creation Successful!!")
except Exception as e:
    print("Table Creation Failed with Error:", {e})

Table Creation Successful!!


In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_song_details (sessionId, itemInSession, artist, song, length) VALUES"
        query = query + "(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [30]:
result_set = session.execute("""SELECT artist, song, length
                                FROM session_song_details
                                WHERE sessionId = 338
                                  AND itemInSession = 4;""")

for r in result_set:
    print(r)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


## Query 2:
#### Query Requirement: 
Give the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Query Requirement analysis: 
Since the query requires filtering on based `sessionId` and `userid`, primary key must have these columns. Also, since it is required for the rows returned to be sorted with respect to `itemInSession`, it should be part of the primary key as a clustering column.
- PARTITION KEY - `(sessionId, userId)` 
- CLUSTURING COLUMNS - `itemInSession` 
- PRIMARY KEY - `((sessionId, userId), itemInSession))` 

#### SELECT statement: 
```SELECT artist, song, firstName, lastName
FROM user_session_history
WHERE userid = 10
  AND sessionid = 182;```

In [36]:
user_history_create_query = """CREATE TABLE IF NOT EXISTS user_session_history (
                                   sessionId INT,
                                   itemInSession INT,
                                   userId INT,
                                   artist TEXT,
                                   song TEXT,
                                   firstName TEXT,
                                   lastName TEXT,
                                   PRIMARY KEY ((sessionId, userId), itemInSession));
                             """

try:
    session.execute(user_history_create_query)
    print("Table Creation Successful!!")
except Exception as e:
    print("Table Creation Failed with Error:", {e})

Table Creation Successful!!


In [37]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session_history (sessionId, itemInSession, userId, artist, song, firstName, lastName) VALUES"
        query = query + "(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[0], line[9], line[1], line[4]))

In [38]:
result_set = session.execute("""SELECT artist, song, firstName, lastName
                                FROM user_session_history
                                WHERE userid = 10
                                  AND sessionid = 182;""")

for r in result_set:
    print(r)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Query 3:
#### Query Requirement: 
Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Query Requirement analysis: 
Since the query requires filtering on song title (`song`), primary key must have the column. Also, since it is required to return all the users who have listened to a song, the `userId` should be part of the primary key as a clustering column as well.
- PARTITION KEY - `song` 
- CLUSTURING COLUMNS - `userId` 
- PRIMARY KEY - `(song, userId)` 

#### SELECT statement: 
```SELECT firstName, lastName
FROM song_listeners
WHERE song = 'All Hands Against His Own';```

In [39]:
song_listeners_create_query = """CREATE TABLE IF NOT EXISTS song_listeners (
                                     userId INT,
                                     song TEXT,
                                     firstName TEXT,
                                     lastName TEXT,
                                     PRIMARY KEY (song, userId));
                              """

try:
    session.execute(song_listeners_create_query)
    print("Table Creation Successful!!")
except Exception as e:
    print("Table Creation Failed with Error:", {e})

Table Creation Successful!!


In [40]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_listeners (userId, song, firstName, lastName) VALUES"
        query = query + "(%s, %s, %s, %s)"
        session.execute(query, (int(line[10]), line[9], line[1], line[4]))

In [41]:
result_set = session.execute("""SELECT firstName, lastName
                                FROM song_listeners
                                WHERE song = 'All Hands Against His Own'""")

for r in result_set:
    print(r)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [42]:
try:
    session.execute("DROP KEYSPACE IF EXISTS song_plays")
    print("Keyspace deleted successfully!!")
except Exception as e:
    print("Keyspace deletion failed with Error:", {e})

Keyspace deleted successfully!!


### Close the session and cluster connection¶

In [43]:
session.shutdown()
cluster.shutdown()